In [12]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ConstantKernel as C
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

steel_data = pd.read_csv("steel.csv")

# Features of the dataset, all cols bar the last
features = steel_data.values[:, :-1]
# Ground truths, the last column
ground_truths = steel_data.values[:, -1]

kf = KFold(n_splits=10, shuffle=False)

model = GaussianProcessRegressor(random_state=4)
for i, (train_index, test_index) in enumerate(kf.split(features)):
    print(f"Fold {i+1}")

    # Features and ground truths for the ith fold
    training_features, test_features = features[train_index], features[test_index]
    training_ground_truths, test_ground_truths = ground_truths[train_index], ground_truths[test_index]

    model.fit(training_features, training_ground_truths)

    prediction = model.predict(test_features)

    r2 = r2_score(test_ground_truths, prediction)
    print(f"R2 Score: {r2}")

    mse = mean_squared_error(test_ground_truths, prediction)
    print(f"Mean Squared Error: {mse}")

param_grid={
    "alpha":[1e-10, 0.01, 0.1, 1, 10],
    "kernel": [
        RBF(length_scale=1.0),
        Matern(length_scale=1.0, nu=1.5),
        RationalQuadratic(length_scale=1.0, alpha=0.5),
        C(constant_value=1.0)
    ]}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring="r2", n_jobs=-1)

grid_search.fit(features, ground_truths)

print(grid_search.best_params_)

Fold 1
R2 Score: -3.809530139643633
Mean Squared Error: 20167.44191878678
Fold 2
R2 Score: -4.521334927005811
Mean Squared Error: 26129.972021292884
Fold 3
R2 Score: -3.615384314800444
Mean Squared Error: 31384.059713151986
Fold 4
R2 Score: -3.667184355698775
Mean Squared Error: 28257.584264482113
Fold 5
R2 Score: -3.6855922461579524
Mean Squared Error: 23575.167520567207
Fold 6
R2 Score: -5.692211004729561
Mean Squared Error: 52048.962193312414
Fold 7
R2 Score: -6.7866640996996
Mean Squared Error: 32426.517319265462
Fold 8
R2 Score: -6.137104891199387
Mean Squared Error: 42822.2969277314
Fold 9
R2 Score: -5.6007333815311195
Mean Squared Error: 74852.90165144335
Fold 10
R2 Score: -9.000939104882836
Mean Squared Error: 84511.1743581762
{'alpha': 1, 'kernel': RBF(length_scale=1)}
